# Testing Different Configurations and Solutions for Parking Management

## Requirements to Run the Notebook

To successfully execute this notebook, you will need the following models for each test:

### Test 1:
- **Required models:**
  - `yolov8n-visdrone.pt`
  - `yolo11n-parking.pt`

### Test 2:
- **Required models:**
  - `yolov8n-visdrone.pt`
  - `yolo11n-detect-parking.pt`

### Test 3:
- **Required models:**
  - `yolov8n-visdrone.pt`
  - `yolo11n-detect-parking.pt`

## How to Obtain Each Model

1. **`yolov8n-visdrone.pt`**  
   This model can be downloaded directly from the following link:  
   [Download yolov8n-visdrone.pt](https://huggingface.co/mshamrai/yolov8n-visdrone)

2. **`yolo11n-parking.pt`**  
   This model needs to be trained by yourself. You can follow the instructions in the notebook **`training_model_world_yolo`**, which provides a step-by-step guide for training this model.

3. **`yolo11n-detect-parking.pt`**  
   This model also requires training on your part. You can do this by following the guide in the **`training_model_dt`** notebook, which includes detailed instructions for training the model.



## Test 1

After training a model that detects sets of parking spaces, we will try to separate them into individual parking spaces using the average size of the cars detected in the video.

In [1]:
import cv2
from ultralytics import solutions
import json
from ultralytics import YOLO
import numpy as np

#---------------------------------------- CREATION OF PARKING SPOTS -----------------------------------------------------------
def get_average_car_size(car_predictions):
    """
    Calculates the average size of cars based on predictions.
    car_predictions: List of bounding boxes of cars, format [x1, y1, x2, y2]
    """
    car_widths = []
    car_heights = []
    
    for box in car_predictions:
        # Calculate the width and height of each car
        width = box[2] - box[0]  # Width: x2 - x1
        height = box[3] - box[1]  # Height: y2 - y1
        # Consider both width and height depending on the car's orientation
        car_widths.append(max(width, height))  # Take the larger value as "width"
        car_heights.append(min(width, height))  # Take the smaller value as "height"

    # Calculate the averages
    avg_car_width = np.mean(car_widths) if car_widths else 0
    avg_car_height = np.mean(car_heights) if car_heights else 0
    
    return avg_car_width, avg_car_height

def divide_parking_area(parking_box, avg_car_width, avg_car_height):
    """
    Divides a large parking area based on the average car size.
    parking_box: Bounding box of the large parking area [x1, y1, x2, y2]
    avg_car_width: Average width of cars
    avg_car_height: Average height of cars
    """
    # If we don't have average car size, we cannot divide correctly
    if avg_car_width == 0 or avg_car_height == 0:
        return [parking_box]  # Return the entire area if there are no cars

    parking_x1, parking_y1, parking_x2, parking_y2 = parking_box
    parking_width = parking_x2 - parking_x1
    parking_height = parking_y2 - parking_y1

    if parking_width > parking_height:
        avg_car_width, avg_car_height = avg_car_height, avg_car_width

    # Calculate the number of spots in both directions (X and Y)
    num_spots_x = int(parking_width // avg_car_width)
    num_spots_y = int(parking_height // avg_car_height)

    spots = []

    total_width_spots = num_spots_x * avg_car_width
    total_height_spots = num_spots_y * avg_car_height

    # Calculate the free space on the left/right and top/bottom
    space_left_right = (parking_width - total_width_spots) / 2
    space_top_bottom = (parking_height - total_height_spots) / 2
    
    # Create parking spots based on the calculated number
    for i in range(num_spots_y):
        for j in range(num_spots_x):
            x = parking_x1 + space_left_right + j * avg_car_width
            y = parking_y1 + space_top_bottom + i * avg_car_height
            w = avg_car_width
            h = avg_car_height
            spots.append([x, y, x + w, y + h])

    return spots


def process_parking_lots(frame_parking, model_parking, model_car):
    # Step 1: Detect parking spots
    results = model_parking.predict(frame_parking)
    parking_predictions = results[0].boxes.xyxy.cpu().numpy()

    # Step 2: Detect cars
    car_results = model_car.predict(frame_parking)
    car_predictions = car_results[0].boxes.xyxy.cpu().numpy()

    # Step 3: Calculate the average size of cars
    avg_car_width, avg_car_height = get_average_car_size(car_predictions)

    # Step 4: Divide large parking spots into small spots based on the size of the cars
    all_spots = []
    for parking_box in parking_predictions:
        spots = divide_parking_area(parking_box, avg_car_width, avg_car_height)
        all_spots.extend(spots)

    return all_spots

cap2 = cv2.VideoCapture("../videos/parking4.mp4")
ret, frame_parking = cap2.read()

parking_spots0 = process_parking_lots(frame_parking, YOLO("../app/models/yolo11n-parking.pt"), YOLO("../app/models/yolov8n-visdrone.pt"))
parking_spots = []
for bbox in parking_spots0:
    x1, y1, x2, y2 = bbox
    polygon = [[int(x1), int(y1)], [int(x2), int(y1)], [int(x2), int(y2)], [int(x1), int(y2)]]
    parking_spots.append({"points": polygon})


bounding_boxes = json.dumps(parking_spots, indent=4)
with open("parking_spots.json", "w") as json_file:
    json_file.write(bounding_boxes)
#---------------------------------------- CREATION OF PARKING SPOTS -----------------------------------------------------------

parking_manager = solutions.ParkingManagement(model="../app/models/yolov8n-visdrone.pt", json_file="parking_spots.json")

# Video capture
cap = cv2.VideoCapture("../videos/parking4.mp4")
assert cap.isOpened(), "Error reading the video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

while cap.isOpened():
    ret, im0 = cap.read()
    if not ret:
        break 
    
    im0 = parking_manager.process_data(im0)
    
    cv2.imshow("Parking Management", im0)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 12 parking_spots, 48.0ms
Speed: 2.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pedestrian, 81 cars, 4 vans, 40.9ms
Speed: 2.0ms preprocess, 40.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Ultralytics Solutions:  {'region': None, 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': 'parking_spots.json', 'model': '../app/models/yolov8n-visdrone.pt'}

0: 384x640 80 cars, 3 vans, 41.0ms
Speed: 2.0ms preprocess, 41.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 80 cars, 3 vans, 41.0ms
Speed: 1.0ms preprocess, 41.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 80 cars, 3 vans, 39.0ms
Speed: 1.0ms preprocess, 39.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 80 cars, 3 vans, 47.0ms
Speed: 2.0ms preprocess, 4

### Results

In some cases, the parking spots are not correctly divided. It runs at low FPS.

## Test 2

In this test, we tried to detect the parking spots individually.

In [1]:
import cv2
import numpy as np
from collections import defaultdict
from ultralytics import YOLO

# Variables to track parking spots
available_spots = set()  # Set of indices for available parking spots
occupied_spots = set()  # Set of indices for occupied parking spots
confidence_threshold = 0.70

def detect_objects(frame, model, threshold):
    """
    Function that uses the YOLO model to detect objects in a frame.
    """
    # Use YOLO to predict on the frame
    detections = model(frame)  # model is assumed to return a Results object
    
    # Get bounding box coordinates, confidence scores, and class indices
    boxes = detections[0].boxes.xyxy.cpu().numpy()  # Bounding box coordinates [x1, y1, x2, y2]
    confidences = detections[0].boxes.conf.cpu().numpy()  # Confidence scores for each detection
    class_indices = detections[0].boxes.cls.cpu().numpy()  # Predicted class indices
    class_names = detections[0].names  # Dictionary mapping class IDs to class names

    # Filter detections based on the confidence threshold
    filtered_boxes = []
    filtered_class_indices = []
    filtered_confidences = []
    
    for box, confidence, class_idx in zip(boxes, confidences, class_indices):
        if confidence >= threshold:
            filtered_boxes.append(box)
            filtered_class_indices.append(class_idx)
            filtered_confidences.append(confidence)
    
    return filtered_boxes, filtered_class_indices, filtered_confidences, class_names

def is_car_in_parking_spot(car_bbox, parking_bbox):
    """
    Function to check if a car is within a parking spot.
    """
    cx, cy = get_centroid(car_bbox)
    px1, py1, px2, py2 = parking_bbox

    # Ensure that the parking box coordinates are properly ordered
    if px1 > px2:
        px1, px2 = px2, px1
    if py1 > py2:
        py1, py2 = py2, py1

    # Check if the car's centroid is inside the parking spot's bounding box
    if px1 <= cx <= px2 and py1 <= cy <= py2:
        return True
    return False

def get_centroid(bbox):
    """
    Calculate the centroid of a bounding box.
    """
    x1, y1, x2, y2 = bbox
    cx = (x1 + x2) / 2
    cy = (y1 + y2) / 2
    return cx, cy

def update_parking_status(cars, parkings):
    """
    Function to update the status of parking spots (available or occupied).
    """
    global available_spots, occupied_spots
    
    # Loop through each parking spot
    for idx, parking in enumerate(parkings):
        # Check if any car is inside the parking spot
        parking_occupied = False
        for car in cars:
            if is_car_in_parking_spot(car, parking):
                parking_occupied = True
                break

        # If the parking spot is occupied
        if parking_occupied:
            if idx not in occupied_spots:
                occupied_spots.add(idx)
            if idx in available_spots:
                available_spots.remove(idx)
        else:
            if idx not in available_spots:
                available_spots.add(idx)
            if idx in occupied_spots:
                occupied_spots.remove(idx)

def process_frame(frame, model, threshold):
    """
    Process a frame to detect cars and update the status of parking spots.
    """
    # Detect objects in the frame using the model with the confidence threshold
    boxes, class_indices, confidences, class_names = detect_objects(frame, model, threshold)
    
    # Filter detections to get cars and parking spots
    cars = []
    for box, class_idx in zip(boxes, class_indices):
        if class_names[int(class_idx)] == 'car':
            cars.append(box)
    
    parkings = []
    for box, class_idx in zip(boxes, class_indices):
        if class_names[int(class_idx)] in ['parking-spot', 'parking-spot-disabled']:
            parkings.append(box)

    
    # Convert parking spot coordinates to a format suitable for comparison
    parking_bboxes = []
    for d in parkings:
        parking_bboxes.append((d[0], d[1], d[2], d[3]))
    
    # Update the parking status
    update_parking_status(cars, parking_bboxes)
    
    return available_spots, occupied_spots, boxes, class_indices, confidences, class_names

def draw_bboxes(frame, boxes, class_indices, confidences, class_names, available_spots, occupied_spots):
    """
    Draw bounding boxes on the frame for each detected object.
    """
    index_parking = 0
    for i, (box, class_idx, confidence) in enumerate(zip(boxes, class_indices, confidences)):
        x1, y1, x2, y2 = box
        label = class_names[int(class_idx)]
        
        # Determine the color based on the type of object
        if label == 'car':
            color = (255, 0, 0)
        elif label in ['parking-spot', 'parking-spot-disabled']:  # Red for occupied, Green for free parking spots
            if index_parking in occupied_spots:
                color = (0, 0, 255)
            else:
                color = (0, 255, 0)
            index_parking += 1
        else:
            color = (255, 255, 255)

        # Draw rectangle around the detected object
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

    # Display the number of available and occupied spots
    available_count = len(available_spots)
    occupied_count = len(occupied_spots)
    cv2.putText(frame, f"Available: {available_count}", (550, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Occupied: {occupied_count}", (550, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    return frame


# Example usage for video processing
video_path = "../videos/parking4.mp4"
model = YOLO("../app/models/yolo11n-detect-parking.pt")

cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading the video file"

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process the frame and update parking status with confidence threshold
    available_spots, occupied_spots, boxes, class_indices, confidences, class_names = process_frame(frame, model, confidence_threshold)

    # Draw bounding boxes and labels with confidence
    frame = draw_bboxes(frame, boxes, class_indices, confidences, class_names, available_spots, occupied_spots)

    # Display the frame with the detections
    cv2.imshow('Parking Detection', frame)
    
    # Exit if the user presses the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


0: 384x640 80 cars, 2 parking-spot-disableds, 113 parking-spots, 71.0ms
Speed: 4.0ms preprocess, 71.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 81 cars, 2 parking-spot-disableds, 112 parking-spots, 44.0ms
Speed: 3.0ms preprocess, 44.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 80 cars, 1 parking-spot-disabled, 111 parking-spots, 49.0ms
Speed: 2.0ms preprocess, 49.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 80 cars, 1 parking-spot-disabled, 111 parking-spots, 45.0ms
Speed: 2.0ms preprocess, 45.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 80 cars, 2 parking-spot-disableds, 111 parking-spots, 43.0ms
Speed: 2.0ms preprocess, 43.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 79 cars, 2 parking-spot-disableds, 111 parking-spots, 40.0ms
Speed: 2.0ms preprocess, 40.0ms inference, 1.0ms postprocess per image at shape

### Results

From what we can see, the model does a good job detecting the parking spots, although it misses some. The model fails to detect roughly half of the vehicles.

## Test 3

In this test, we aim to use the model from test2 (yolo11n-detect-parking.pt) to detect the parking spots, and the yolov8n-visdrone model to accurately detect the cars.

In [1]:
import cv2
import numpy as np
from collections import defaultdict
from ultralytics import YOLO

# Variables to track parking spots
available_spots = set()  # Set of indices for available parking spots
occupied_spots = set()  # Set of indices for occupied parking spots
confidence_threshold = 0.70

def detect_objects(frame, model, threshold):
    """
    Function that uses the YOLO model to detect objects in a frame.
    """

    # Use YOLO to predict on the frame
    detections = model(frame)  # model is assumed to return a Results object

    
    # Get bounding box coordinates, confidence scores, and class indices
    boxes = detections[0].boxes.xyxy.cpu().numpy()  # Bounding box coordinates [x1, y1, x2, y2]
    confidences = detections[0].boxes.conf.cpu().numpy()  # Confidence scores for each detection
    class_indices = detections[0].boxes.cls.cpu().numpy()  # Predicted class indices
    class_names = detections[0].names  # Dictionary mapping class IDs to class names

    # Filter detections based on the confidence threshold
    filtered_boxes = []
    filtered_class_indices = []
    filtered_confidences = []
    
    for box, confidence, class_idx in zip(boxes, confidences, class_indices):
        if confidence >= threshold:
            filtered_boxes.append(box)
            filtered_class_indices.append(class_idx)
            filtered_confidences.append(confidence)
    
    return filtered_boxes, filtered_class_indices, filtered_confidences, class_names

def is_car_in_parking_spot(car_bbox, parking_bbox):
    """
    Function to check if a car is within a parking spot.
    """
    cx, cy = get_centroid(car_bbox)
    px1, py1, px2, py2 = parking_bbox

    # Ensure that the parking box coordinates are properly ordered
    if px1 > px2:
        px1, px2 = px2, px1
    if py1 > py2:
        py1, py2 = py2, py1

    # Check if the car's centroid is inside the parking spot's bounding box
    if px1 <= cx <= px2 and py1 <= cy <= py2:
        return True
    return False

def get_centroid(bbox):
    """
    Calculate the centroid of a bounding box.
    """
    x1, y1, x2, y2 = bbox
    cx = (x1 + x2) / 2
    cy = (y1 + y2) / 2
    return cx, cy

def update_parking_status(cars, parkings):
    """
    Function to update the status of parking spots (available or occupied).
    """
    global available_spots, occupied_spots
    
    # Loop through each parking spot
    for idx, parking in enumerate(parkings):
        # Check if any car is inside the parking spot
        parking_occupied = False
        for car in cars:
            if is_car_in_parking_spot(car, parking):
                parking_occupied = True
                break

        # If the parking spot is occupied
        if parking_occupied:
            if idx not in occupied_spots:
                occupied_spots.add(idx)
            if idx in available_spots:
                available_spots.remove(idx)
        else:
            if idx not in available_spots:
                available_spots.add(idx)
            if idx in occupied_spots:
                occupied_spots.remove(idx)

def process_frame(frame, model, parkings_predictions=[], threshold=0.5):
    """
    Process a frame to detect cars and update the status of parking spots.
    """
    boxes_parkings, class_indices_parkings, confidences_parkings, class_names_parkings = parkings_predictions
    
    parkings = []
    # Filter detections to get cars and parking spots
    for box, class_idx in zip(boxes_parkings, class_indices_parkings):
        if class_names_parkings[int(class_idx)] in ['parking-spot', 'parking-spot-disabled']:
            parkings.append(box)

    model_vehicles = YOLO("../app/models/yolov8n-visdrone.pt")
    boxes_vehicles, class_indices_vehicles, confidences_vehicles, class_names_vehicles = detect_objects(frame, model_vehicles, threshold)
    cars = []
    for box, class_idx in zip(boxes_vehicles, class_indices_vehicles):
        if class_names_vehicles[int(class_idx)] in ['car', 'van', 'truck']:
            cars.append(box)
    
    # Convert parking spot coordinates to a format suitable for comparison
    parking_bboxes = []
    for d in parkings:
        parking_bboxes.append((d[0], d[1], d[2], d[3]))
    
    # Update the parking status
    update_parking_status(cars, parking_bboxes)
    return available_spots, occupied_spots, boxes_vehicles, class_indices_vehicles, confidences_vehicles, class_names_vehicles

def draw_bboxes_parking_spots(frame, boxes, class_indices, confidences, class_names, available_spots, occupied_spots):
    """
    Draw bounding boxes on the frame for each detected object.
    """
    for i, (box, class_idx, confidence) in enumerate(zip(boxes, class_indices, confidences)):
        x1, y1, x2, y2 = box
        label = class_names[int(class_idx)]
        
        # Determine the color based on the type of object
        if label in ['parking-spot', 'parking-spot-disabled']:  # Red for occupied, Green for free parking spots
            if i in occupied_spots:
                color = (0, 0, 255)  # Red for occupied parking spot
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
            else:
                color = (0, 255, 0)  # Green for available parking spot
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

    # Display the number of available and occupied spots
    available_count = len(available_spots)
    occupied_count = len(occupied_spots)
    cv2.putText(frame, f"Available: {available_count}", (550, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Occupied: {occupied_count}", (550, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    return frame

def draw_bboxes_vehicles(frame, boxes, class_indices, confidences, class_names, available_spots, occupied_spots):
    """
    Draw bounding boxes on the frame for each detected object.
    Uses different colors for cars, occupied parking spots, and free parking spots.
    """
    for i, (box, class_idx, confidence) in enumerate(zip(boxes, class_indices, confidences)):
        x1, y1, x2, y2 = box
        label = class_names[int(class_idx)]
        
        # Determine the color based on the type of object
        if label == 'car' or 'van' or 'truck':  # Blue for cars
            color = (255, 0, 0)  # Blue color for cars
            # Draw rectangle around the detected object
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
  
    return frame

cap2 = cv2.VideoCapture("../videos/parking4.mp4")
assert cap2.isOpened(), "Error reading the video file"
ret, frame_parking = cap2.read()
model = YOLO("../app/models/yolo11n-detect-parking.pt")

parkings_predictions = detect_objects(frame_parking, model, confidence_threshold)

boxes_parkings = []
class_indices_parkings = []
confidences_parkings = []
for box, class_idx, confidence in zip(parkings_predictions[0], parkings_predictions[1], parkings_predictions[2]):
    if parkings_predictions[3][int(class_idx)] in ['parking-spot', 'parking-spot-disabled']:
        boxes_parkings.append(box)
        class_indices_parkings.append(class_idx)
        confidences_parkings.append(confidence)

# Example usage for video processing
video_path = "../videos/parking4.mp4"
model = YOLO("../app/models/yolo11n-detect-parking.pt")

cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading the video file"

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process the frame and update parking status with confidence threshold
    available_spots, occupied_spots, boxes, class_indices, confidences, class_names = process_frame(frame, model, parkings_predictions, confidence_threshold)

    # Draw bounding boxes and labels with confidence
    frame = draw_bboxes_parking_spots(frame, boxes_parkings, class_indices_parkings, confidences_parkings, parkings_predictions[3], available_spots, occupied_spots)
    frame = draw_bboxes_vehicles(frame, boxes, class_indices, confidences, class_names, available_spots, occupied_spots)

    # Display the frame with the detections
    cv2.imshow('Parking Detection', frame)
    
    # Exit if the user presses the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


0: 384x640 80 cars, 2 parking-spot-disableds, 113 parking-spots, 54.0ms
Speed: 2.0ms preprocess, 54.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pedestrian, 81 cars, 4 vans, 42.0ms
Speed: 2.0ms preprocess, 42.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 81 cars, 4 vans, 60.0ms
Speed: 7.0ms preprocess, 60.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pedestrian, 82 cars, 5 vans, 42.0ms
Speed: 2.0ms preprocess, 42.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pedestrian, 82 cars, 5 vans, 37.0ms
Speed: 2.0ms preprocess, 37.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pedestrian, 82 cars, 5 vans, 43.0ms
Speed: 1.0ms preprocess, 43.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 82 cars, 4 vans, 34.0ms
Speed: 2.0ms preprocess, 34.0ms inference, 1.0ms postprocess per image at

### Results

In this case, it correctly detects most of the vehicles, along with detecting a large number of parking spots. The problem is that it runs at a low FPS.